In [1]:
import numpy as np
import json
import string
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [22]:
#data = pd.read_csv('output_landmarks.csv', header=None)

In [2]:
def convert_to_json(string):
    try:
        string = string.replace("'", "\"")
        string = string.replace("\"\"\"", "\"'\"")
        return json.loads(string)
    except ValueError:
        print(string)
        return None

In [3]:
#df = pd.read_csv(r"C:\Users\user\Desktop\Courses\2024Winter\COMP313\Code\output_landmarks.csv", header=None, names=["index", "landmark", "target"])
df = pd.read_csv(r"C:\Users\emrey\Desktop\Centennial\FOURTH SEMESTER\Software Development\output_landmarks.csv", header=None, names=["index", "landmark", "target"])


In [4]:
df.head(10)

,index,landmark,target
0,0,"[[[0.3720352351665497, 0.5708810091018677, 5.6...","['d', 'c']"
1,1,"[[[0.3651290535926819, 0.5668753981590271, 3.8...","['e', 'd']"
2,2,"[[[0.5721478462219238, 0.879878580570221, -1.6...","['a', 's', 'l', 'i', 'z', 'e', 'd']"
3,3,"[[[0.3897995352745056, 0.5905594825744629, -1....","['a', 's', 'l', ' ', 'l', 'i', 't', 'e', 'r', ..."
4,4,"[[[0.34600889682769775, 0.6220497488975525, 1....","['a', 'l', 'i', 'z', 'e', 'd']"
5,5,"[[], [], [[0.5789754390716553, 0.8081677556037...","['a', 's', 'l']"
6,6,"[[[0.5133020281791687, 0.6245256662368774, 1.7...","['a', 's', 'l']"
7,7,"[[[0.49991101026535034, 0.8499583005905151, -3...","['e', 'd']"
8,8,"[[[0.36749503016471863, 0.6053090691566467, 2....","['a', 's', 'l']"
9,9,"[[[0.63340163230896, 0.7506250143051147, -2.55...","['v', 'i', 'd', 'e', 'o']"


In [5]:
df['target'] = df['target'].apply(convert_to_json)

In [6]:
df['landmark'] = df['landmark'].apply(json.loads)

# Display unique values on df['target']

In [7]:
df.head()
# Getting unique values for each row's target
df['unique_target_per_row'] = df['target'].apply(lambda x: list(set(x)))

# Getting unique values of 'target' among all rows
unique_values_among_all_rows = list(set().union(*df['target']))

# Displaying unique values per row
print(df['unique_target_per_row'].head(10))

# Displaying unique values among all rows
print(unique_values_among_all_rows)
print(len(unique_values_among_all_rows))

0                         [c, d]
1                         [e, d]
2          [e, i, a, l, z, s, d]
3    [e, t, i,  , a, l, u, s, r]
4             [e, i, a, l, z, d]
5                      [l, a, s]
6                      [l, a, s]
7                         [e, d]
8                      [l, a, s]
9                [e, i, o, d, v]
Name: unique_target_per_row, dtype: object
['e', 'j', '&', 'z', 'g', 'p', 'i', ' ', "'", '@', 'o', 's', 'd', 'r', 'v', 'x', 'n', 'q', 'a', 'm', 'f', '.', 'c', 'w', 'h', 't', 'y', 'b', 'l', 'u', 'k']
31


# Dropping the rows where labels contains non alphabets

In [8]:
import pandas as pd

# Assuming df is your DataFrame and 'target' is the column you're working with

# Define a function to check for non-alphabetical characters in a list
def contains_non_alpha(lst):
    return any(not char.isalpha() for char in ''.join(lst))


# Apply the function to the 'target' column to identify rows with non-alphabetical characters
rows_with_non_alpha = df['target'].apply(contains_non_alpha)

# Count rows containing non-alphabetical characters
count_non_alpha_rows = rows_with_non_alpha.sum()

# Drop these rows from the DataFrame
# After dropping rows
df = df[~rows_with_non_alpha].reset_index(drop=True)


print(f"Rows containing non-alphabetical characters: {count_non_alpha_rows}")
# df_clean will have rows removed that contained non-alphabetical characters in 'target'

# Getting unique values of 'target' among all rows
unique_values_among_all_rows2 = list(set().union(*df['target']))


# Displaying unique values among all rows
print(unique_values_among_all_rows2)
print(len(unique_values_among_all_rows2))

Rows containing non-alphabetical characters: 1114
['e', 'j', 'z', 'g', 'p', 'i', 'o', 's', 'd', 'r', 'v', 'x', 'n', 'q', 'a', 'm', 'f', 'c', 'w', 'h', 't', 'y', 'b', 'l', 'u', 'k']
26


# investigation

In [9]:
def round_nested_list(nested_list, decimals):
    def round_item(item):
        if isinstance(item, list):  # Check if the item is a list itself
            return [round_item(sub_item) for sub_item in item]  # Recurse if item is a list
        elif isinstance(item, (int, float)):  # Check if the item is numeric
            return round(item, decimals)  # Round if item is a number
        else:
            return item  # Return the item unchanged if it's not a list or number
    
    return [round_item(item) for item in nested_list]

# Assuming df['landmark'] contains nested lists that need rounding
# Apply the rounding function
df['rounded_landmark'] = df['landmark'].apply(lambda x: round_nested_list(x, 2))

print(df['rounded_landmark'].head(10))


0    [[[0.37, 0.57, 0.0], [0.39, 0.55, -0.01], [0.4...
1    [[[0.37, 0.57, 0.0], [0.39, 0.54, -0.02], [0.4...
2    [[[0.57, 0.88, -0.0], [0.55, 0.84, -0.0], [0.5...
3    [[[0.35, 0.62, 0.0], [0.37, 0.57, -0.0], [0.38...
4    [[], [], [[0.58, 0.81, -0.0], [0.57, 0.75, 0.0...
5    [[[0.51, 0.62, 0.0], [0.53, 0.58, 0.0], [0.56,...
6    [[[0.5, 0.85, -0.0], [0.48, 0.78, 0.0], [0.47,...
7    [[[0.37, 0.61, 0.0], [0.39, 0.57, 0.0], [0.41,...
8    [[[0.63, 0.75, -0.0], [0.61, 0.69, -0.01], [0....
9    [[[0.36, 0.63, -0.0], [0.39, 0.62, -0.01], [0....
Name: rounded_landmark, dtype: object


In [31]:
# Assuming 'df' is your DataFrame
# Slice to get the first 100 rows
first_100_rows = df.iloc[:100]

# Specify your desired output path
output_path = r"C:\Users\user\Desktop\Courses\2024Winter\COMP313\Code\first_100_output_landmarks.csv"

# Save to CSV without the index unless you need it
first_100_rows.to_csv(output_path, index=False)


# Normalize features

In [10]:
#print(json.dumps(rounded_landmark.iloc[0], indent=4))
def find_dimensions(obj):
    """Find the dimensions of a nested list or a numpy array."""
    if isinstance(obj, np.ndarray):
        # If the object is a numpy array, return its shape directly.
        return obj.shape
    elif isinstance(obj, list):
        # If the object is a list, compute dimensions recursively as before.
        if not obj:  # Base case: empty list
            return []
        return [len(obj)] + find_dimensions(obj[0])
    else:
        # If the object is neither a list nor a numpy array, return an empty list.
        # Alternatively, you might handle other types or raise an error.
        return []

    
print("df['target']: \n", df['target'].head())    
    
    
    
import pandas as pd

# Initialize variables to track the maximum dimensions
max_dim1 = 0
max_dim2 = 0
max_dim3 = 0

# Function to recursively find the maximum dimensions of a nested list
def update_max_dims(landmark, level=0):
    global max_dim1, max_dim2, max_dim3
    # Check if the current element is a list (and not just a numerical value)
    if isinstance(landmark, list):
        # At the top level, update max_dim1 based on the number of elements (e.g., groups of coordinates)
        if level == 0:
            max_dim1 = max(max_dim1, len(landmark))
        # At the first level of nesting, update max_dim2
        elif level == 1:
            max_dim2 = max(max_dim2, len(landmark))
        # At the second level of nesting (actual coordinates), update max_dim3
        elif level == 2:
            max_dim3 = max(max_dim3, len(landmark))
        
        # Recursively process each element in the current list, incrementing the level
        for sub_landmark in landmark:
            update_max_dims(sub_landmark, level+1)

# Iterate over the 'landmark' column and update maximum dimensions
for landmark in df['landmark']:
    update_max_dims(landmark)

print(f"Maximum dimensions found in df['landmark']: {max_dim1}, {max_dim2}, {max_dim3}")
max_dim1 = 0
max_dim2 = 0
max_dim3 = 0
for target in df['target']:
    update_max_dims(target)

print(f"Maximum dimensions found in df['target']: {max_dim1}, {max_dim2}, {max_dim3}")


df['target']: 
 0                   [d, c]
1                   [e, d]
2    [a, s, l, i, z, e, d]
3       [a, l, i, z, e, d]
4                [a, s, l]
Name: target, dtype: object
Maximum dimensions found in df['landmark']: 125, 21, 3
Maximum dimensions found in df['target']: 17, 0, 0


In [11]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

def normalize_landmarks(landmarks_seq):
    # This will hold the normalized sequences
    normalized_seq = []
    
    for matrix in landmarks_seq:
        # Ensure the matrix is not empty to avoid errors during scaling
        if matrix:  # Checks if the matrix is not empty
            matrix_np = np.array(matrix)
            # Normalize the [21x3] matrix
            # Note: scaler.fit_transform expects a 2D array, [samples, features]
            # Reshape is used to ensure the matrix is 2D and to revert it back to original shape after scaling
            scaled_matrix = scaler.fit_transform(matrix_np.reshape(-1, 3)).reshape(21, 3)
            normalized_seq.append(scaled_matrix)
        else:
            # Handle empty matrices by appending an array of zeros or another placeholder
            # This keeps the temporal structure consistent
            normalized_seq.append(np.zeros((21, 3)))
    
    return normalized_seq

# Apply the normalization function to each sequence in 'landmarks'
df['normalized_landmarks'] = df['landmark'].apply(normalize_landmarks)


# Display the structure of the first 5 elements in 'normalized_landmarks'
for i in range(5):
    # Access the ith element in 'normalized_landmarks'
    landmarks_seq = df['normalized_landmarks'].iloc[i]
    # Since landmarks_seq is a list of arrays, let's print out the number of frames
    # and the shape of the first frame (if available) to understand its structure
    num_frames = len(landmarks_seq)
    first_frame_shape = landmarks_seq[0].shape if num_frames > 0 else "No frames"
    print(f"Sequence {i} has {num_frames} frames; First frame shape: {first_frame_shape}")


Sequence 0 has 19 frames; First frame shape: (21, 3)
Sequence 1 has 13 frames; First frame shape: (21, 3)
Sequence 2 has 42 frames; First frame shape: (21, 3)
Sequence 3 has 32 frames; First frame shape: (21, 3)
Sequence 4 has 15 frames; First frame shape: (21, 3)


# Padding

# Padding  df['normalized_landmarks']

In [12]:

# Step 2: Pad the scaled landmark data
max_frames = 125  # Target number of frames for each sequence

def pad_sequence(sequence):
    # Determine the number of frames to pad
    num_frames_to_pad = max_frames - len(sequence)
    
    # Create frames of zeros to be appended
    # Each frame is a (21, 3) array of zeros
    padding_frames = [np.zeros((21, 3)) for _ in range(num_frames_to_pad)]
    
    # Append the padding frames to the sequence
    padded_sequence = sequence + padding_frames
    
    return padded_sequence

# Apply the padding function to each sequence in 'normalized_landmarks'
df['padded_normalized_landmarks'] = df['normalized_landmarks'].apply(pad_sequence)



# Verify the padding for the first 5 sequences
for i in range(5):
    num_frames = len(df['padded_normalized_landmarks'].iloc[i])
    print(f"Sequence {i} after padding has {num_frames} frames.")
    first_frame_shape = landmarks_seq[0].shape if num_frames > 0 else "No frames"
    print(f"Sequence {i} has {num_frames} frames; First frame shape: {first_frame_shape}")



Sequence 0 after padding has 125 frames.
Sequence 0 has 125 frames; First frame shape: (21, 3)
Sequence 1 after padding has 125 frames.
Sequence 1 has 125 frames; First frame shape: (21, 3)
Sequence 2 after padding has 125 frames.
Sequence 2 has 125 frames; First frame shape: (21, 3)
Sequence 3 after padding has 125 frames.
Sequence 3 has 125 frames; First frame shape: (21, 3)
Sequence 4 after padding has 125 frames.
Sequence 4 has 125 frames; First frame shape: (21, 3)


# Padding df['target']

In [13]:
max_length_target = 17  # The maximum length of target sequences

def pad_target_sequence(target_seq):
    # Calculate the number of elements to pad
    num_elements_to_pad = max_length_target - len(target_seq)
    
    # Create the padding elements
    padding_elements = [0] * num_elements_to_pad  # Use 0 or another appropriate padding token
    
    # Append the padding elements to the target sequence
    padded_target_seq = target_seq + padding_elements
    
    return padded_target_seq

# Apply the padding function to each target sequence in 'target'
df['padded_target'] = df['target'].apply(pad_target_sequence)

# Verify the first 5 sequences in 'padded_target'
for i in range(5):
    seq_length = len(df['padded_target'].iloc[i])
    print(f"Sequence {i} in 'padded_target' has {seq_length} elements.")


Sequence 0 in 'padded_target' has 17 elements.
Sequence 1 in 'padded_target' has 17 elements.
Sequence 2 in 'padded_target' has 17 elements.
Sequence 3 in 'padded_target' has 17 elements.
Sequence 4 in 'padded_target' has 17 elements.


# Label encoding

In [14]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

# Flatten the list of all sequences to get a list of all labels
# Assuming df['padded_target'] contains sequences of labels
all_labels = [label for sequence in df['padded_target'] for label in sequence]

# Initialize the LabelEncoder and fit it with the flattened list of all labels
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

def encode_sequence(sequence):
    # Transform each element in the sequence using the fitted LabelEncoder
    return [label_encoder.transform([item])[0] for item in sequence]

# Apply the encoding function to each sequence in 'padded_target'
df['encoded_target'] = df['padded_target'].apply(encode_sequence)


# Display the unique labels and their corresponding integer codes
labels_and_codes = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label to integer code mapping:", labels_and_codes)




Label to integer code mapping: {'0': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [15]:
import joblib

# Save the label encoder to a file
joblib.dump(label_encoder, 'label_encoder.joblib')


['label_encoder.joblib']

# Preprocessed features and labels

In [16]:
print(df['padded_normalized_landmarks'].head(10))
print(df['encoded_target'].head(10))

0    [[[-1.2153970288004756, 2.3753629941416357, 2....
1    [[[0.2901805728306801, 2.12861539170463, 2.667...
2    [[[2.6813927528272097, 0.3813247752700488, 2.0...
3    [[[-1.3134348894070909, 2.2045729014403244, 2....
4    [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...
5    [[[-2.4358860944922798, 0.5291534544608159, 1....
6    [[[0.6172452529867535, 2.6189231677172247, 1.5...
7    [[[-2.3207745744526025, 0.7424863477272686, 2....
8    [[[0.2045866255829034, 1.8983832688894848, 2.5...
9    [[[-0.5809574688840651, 0.45024894346511807, 2...
Name: padded_normalized_landmarks, dtype: object
0    [4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [1, 19, 12, 9, 26, 5, 4, 0, 0, 0, 0, 0, 0, 0, ...
3    [1, 12, 9, 26, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0...
4    [1, 19, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
5    [1, 19, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...
6    [5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7    [1, 19, 12,

# dropping z

In [17]:
import numpy as np

# Example DataFrame setup for context
# import pandas as pd
# df = pd.DataFrame({'padded_normalized_landmarks': [np.random.rand(125, 21, 2) for _ in range(5)]})

# Process each sequence to reshape the frames
# Assuming df['padded_normalized_landmarks'] contains your sequences
df['reshaped_landmarks'] = df['padded_normalized_landmarks'].apply(
    lambda seq: np.concatenate(seq, axis=1).reshape(125, -1).tolist()
)
# Now, df['reshaped_landmarks'] contains sequences with shape (125, 42)

# Verifying the "shape" change for the first sequence by checking the length
original_first_seq_length = len(df['padded_normalized_landmarks'].iloc[0])
original_frame_shape = df['padded_normalized_landmarks'].iloc[0][0].shape

reshaped_first_seq_length = len(df['reshaped_landmarks'].iloc[0])
reshaped_frame_shape = len(df['reshaped_landmarks'].iloc[0][0])

print(f"Original first sequence length: {original_first_seq_length}")
print(f"Original frame shape: {original_frame_shape}")
print(f"Reshaped first sequence length: {reshaped_first_seq_length}")
print(f"Reshaped frame shape: {reshaped_frame_shape}")


Original first sequence length: 125
Original frame shape: (21, 3)
Reshaped first sequence length: 125
Reshaped frame shape: 63


In [22]:
len(df['reshaped_landmarks'][0][0])

63

In [19]:
df.head()

,index,landmark,target,unique_target_per_row,rounded_landmark,normalized_landmarks,padded_normalized_landmarks,padded_target,encoded_target,reshaped_landmarks
0,0,"[[[0.3720352351665497, 0.5708810091018677, 5.6...","[d, c]","[c, d]","[[[0.37, 0.57, 0.0], [0.39, 0.55, -0.01], [0.4...","[[[-1.2153970288004756, 2.3753629941416357, 2....","[[[-1.2153970288004756, 2.3753629941416357, 2....","[d, c, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[-1.2153970288004756, 2.3753629941416357, 2.3..."
1,1,"[[[0.3651290535926819, 0.5668753981590271, 3.8...","[e, d]","[d, e]","[[[0.37, 0.57, 0.0], [0.39, 0.54, -0.02], [0.4...","[[[0.2901805728306801, 2.12861539170463, 2.667...","[[[0.2901805728306801, 2.12861539170463, 2.667...","[e, d, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0.2901805728306801, 2.12861539170463, 2.6670..."
2,2,"[[[0.5721478462219238, 0.879878580570221, -1.6...","[a, s, l, i, z, e, d]","[l, i, a, e, s, z, d]","[[[0.57, 0.88, -0.0], [0.55, 0.84, -0.0], [0.5...","[[[2.6813927528272097, 0.3813247752700488, 2.0...","[[[2.6813927528272097, 0.3813247752700488, 2.0...","[a, s, l, i, z, e, d, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 19, 12, 9, 26, 5, 4, 0, 0, 0, 0, 0, 0, 0, ...","[[2.6813927528272097, 0.3813247752700488, 2.09..."
3,4,"[[[0.34600889682769775, 0.6220497488975525, 1....","[a, l, i, z, e, d]","[l, i, a, e, z, d]","[[[0.35, 0.62, 0.0], [0.37, 0.57, -0.0], [0.38...","[[[-1.3134348894070909, 2.2045729014403244, 2....","[[[-1.3134348894070909, 2.2045729014403244, 2....","[a, l, i, z, e, d, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 12, 9, 26, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[-1.3134348894070909, 2.2045729014403244, 2.0..."
4,5,"[[], [], [[0.5789754390716553, 0.8081677556037...","[a, s, l]","[s, l, a]","[[], [], [[0.58, 0.81, -0.0], [0.57, 0.75, 0.0...","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...","[a, s, l, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 19, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.000440961209..."


In [18]:
import numpy as np

# Example DataFrame setup for context
# import pandas as pd
# df = pd.DataFrame({'padded_normalized_landmarks': [np.random.rand(125, 21, 3) for _ in range(5)]})

# Process each sequence to remove the third column
# Assuming df['padded_normalized_landmarks'] contains your sequences
df['padded_normalized_landmarks_21x2'] = df['padded_normalized_landmarks'].apply(
    lambda seq: np.array(seq)[:, :, :2].tolist()
)
# Now, df['padded_normalized_landmarks_21x2'] contains sequences with shape (125, 21, 2)

# Verifying the "shape" change for the first sequence by checking the lengths
original_first_seq_length = len(df['padded_normalized_landmarks'].iloc[0])
original_first_frame_shape = (len(df['padded_normalized_landmarks'].iloc[0][0]), len(df['padded_normalized_landmarks'].iloc[0][0][0]))

modified_first_seq_length = len(df['padded_normalized_landmarks_21x2'].iloc[0])
modified_first_frame_shape = (len(df['padded_normalized_landmarks_21x2'].iloc[0][0]), len(df['padded_normalized_landmarks_21x2'].iloc[0][0][0]))

print(f"Original first sequence length: {original_first_seq_length}")
print(f"Original first frame shape: {original_first_frame_shape}")
print(f"Modified first sequence length: {modified_first_seq_length}")
print(f"Modified first frame shape: {modified_first_frame_shape}")


Original first sequence length: 125
Original first frame shape: (21, 3)
Modified first sequence length: 125
Modified first frame shape: (21, 2)


In [19]:
df['padded_normalized_landmarks_42']=df['padded_normalized_landmarks_21x2'].apply(lambda x : np.array(x))
df['padded_normalized_landmarks_42']=df['padded_normalized_landmarks_42'].apply(lambda x : x.reshape(125,42))

# Data Splitting

In [30]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Assume df['padded_normalized_landmarks'] and df['encoded_target'] are already prepared
X = np.stack(df['padded_normalized_landmarks_42'].values)
y = np.array([to_categorical(sequence, num_classes=27) for sequence in df['encoded_target']])

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)


# LSTM

In [51]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Reshape, Flatten
# import numpy as np

# num_encoder_tokens = 63  # 21 landmarks * 3 coordinates
# num_decoder_tokens = 27  # Including 0-26 (26 letters + 'blank')
# latent_dim = 256  # Latent dimensionality of the encoding space
# # Adjusting configuration to accept the 3D frame shape directly
# num_encoder_tokens = 21 * 3  # Keeping this for clarity, though not directly used below

# # Encoder
# encoder_inputs = Input(shape=(125, 21, 3))  # Adjusted to match the input shape

# # Flatten the spatial dimensions of the frame for the LSTM
# encoder_inputs_flat = TimeDistributed(Flatten())(encoder_inputs)

# encoder = LSTM(latent_dim, return_state=True)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs_flat)
# encoder_states = [state_h, state_c]

# # Decoder setup remains the same
# decoder_inputs = Input(shape=(None, num_decoder_tokens))
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

# # Define the adjusted model
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# LSTM with attention

In [31]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, Flatten, Concatenate, Attention
import numpy as np

num_encoder_tokens = 42  # 21 landmarks * 2 coordinates (Not directly used, but for clarity)
num_decoder_tokens = 27  # Including 0-26 (26 letters + 'blank')
latent_dim = 256  # Latent dimensionality of the encoding space

# Encoder
encoder_inputs = Input(shape=(125, 42), name='encoder_inputs')
encoder_inputs_flat = TimeDistributed(Flatten(), name='encoder_inputs_flat')(encoder_inputs)

# Using return_sequences=True to get all hidden states for attention
encoder_lstm = LSTM(latent_dim, return_state=True, return_sequences=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs_flat)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Attention Layer
attention_layer = Attention(name='attention_layer')
attention_result = attention_layer([decoder_outputs, encoder_outputs])

# Concatenating the attention output and the decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attention_result])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, 125, 42)]            0         []                            
 )                                                                                                
                                                                                                  
 encoder_inputs_flat (TimeD  (None, 125, 42)              0         ['encoder_inputs[0][0]']      
 istributed)                                                                                      
                                                                                                  
 decoder_inputs (InputLayer  [(None, None, 27)]           0         []                            
 )                                                                                            

In [56]:
# Assuming y_train, y_val are one-hot encoded and already split
# Shift target sequences one timestep for decoder input, introducing start token
decoder_input_train = np.zeros_like(y_train)
decoder_input_val = np.zeros_like(y_val)
decoder_input_train[:, 1:, :] = y_train[:, :-1, :]
decoder_input_val[:, 1:, :] = y_val[:, :-1, :]


In [57]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([X_train, decoder_input_train], y_train, batch_size=64, epochs=20, validation_data=([X_val, decoder_input_val], y_val))


Epoch 1/20
59/59 [==============================] - 53s 746ms/step - loss: 1.0857 - accuracy: 0.7484 - val_loss: 1.0145 - val_accuracy: 0.7516
Epoch 2/20
59/59 [==============================] - 40s 682ms/step - loss: 0.8963 - accuracy: 0.7672 - val_loss: 0.9252 - val_accuracy: 0.7690
Epoch 3/20
59/59 [==============================] - 42s 718ms/step - loss: 0.8488 - accuracy: 0.7705 - val_loss: 0.8570 - val_accuracy: 0.7722
Epoch 4/20
59/59 [==============================] - 42s 712ms/step - loss: 0.8051 - accuracy: 0.7760 - val_loss: 0.8391 - val_accuracy: 0.7683
Epoch 5/20
59/59 [==============================] - 42s 718ms/step - loss: 0.7716 - accuracy: 0.7802 - val_loss: 0.9349 - val_accuracy: 0.7649
Epoch 6/20
59/59 [==============================] - 42s 710ms/step - loss: 0.7415 - accuracy: 0.7887 - val_loss: 0.8336 - val_accuracy: 0.7621
Epoch 7/20
59/59 [==============================] - 41s 700ms/step - loss: 0.7108 - accuracy: 0.7968 - val_loss: 0.7233 - val_accuracy: 0.7880

# Save model

In [39]:
model.save('model_sign_2d_dim42.h5')


C:\Users\emrey\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Test evaluation

In [58]:
# Preparing decoder_input_test similar to decoder_input_train and decoder_input_val
decoder_input_test = np.zeros_like(y_test)
decoder_input_test[:, 1:, :] = y_test[:, :-1, :]

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate([X_test, decoder_input_test], y_test, verbose=1)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")


39/39 [==============================] - 2s 47ms/step - loss: 0.6101 - accuracy: 0.8266
Test Loss: 0.610126793384552
Test Accuracy: 0.8266178965568542


In [90]:
decoder_input_data.shape

(1, 1, 27)

In [111]:
import numpy as np


# Assuming num_decoder_tokens is defined as the number of tokens in your decoder vocabulary
num_decoder_tokens = 27  # Including 0-26 (26 letters + 'blank')

# Reshape the data point to match the input shape of the decoder model
decoder_input_data = np.zeros((1, 1, num_decoder_tokens))  # Initialize decoder input data

# Set the first token (start token) to 1
decoder_input_data[0, 0, 0] = 1  # Assuming the first token represents the start token

# Make predictions using the model
decoder_output_data = model.predict(([dummy_array, decoder_input_data]))


1/1 [==============================] - 1s 835ms/step


In [113]:
predicted_labels = np.argmax(decoder_output_data, axis=-1)
predicted_labels

array([[19]], dtype=int64)

In [41]:
decoder_input_test[:,1:,:]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

# Test Predicton presentation 

In [51]:
predicted_sequences[0]

array([[3.07063874e-10, 1.24656010e-08, 6.39067082e-07, 1.31045226e-07,
        8.38511646e-01, 1.40038566e-04, 2.42143142e-06, 7.50711251e-07,
        1.31595769e-07, 1.23739490e-08, 2.77366695e-07, 1.65909587e-04,
        6.11476366e-07, 8.51442110e-06, 3.04915716e-07, 1.49868652e-06,
        3.97147305e-05, 6.88448472e-06, 1.65901525e-04, 2.51985085e-03,
        1.44903504e-04, 1.57305002e-01, 8.78686842e-04, 8.68660118e-06,
        2.25113208e-08, 5.21320730e-11, 9.74789073e-05],
       [7.65537678e-09, 1.15289209e-08, 1.10208564e-09, 1.85441495e-05,
        5.81177846e-06, 9.49103415e-01, 4.35545289e-08, 3.79766227e-08,
        1.37311398e-10, 3.26950862e-06, 1.27080668e-07, 1.57599239e-10,
        1.71655247e-05, 5.82714543e-10, 3.57860859e-11, 5.82137181e-05,
        5.69968028e-09, 8.47796997e-08, 4.49338449e-06, 5.07864617e-02,
        1.71086083e-06, 1.50928159e-07, 4.52895428e-08, 1.10998872e-08,
        8.90828602e-08, 7.88709364e-09, 2.78862643e-07],
       [3.42050754e-03

In [41]:
# Preparing the decoder input for prediction by shifting the y_test sequences
decoder_input_test = np.zeros_like(y_test)
decoder_input_test[:, 1:, :] = y_test[:, :-1, :]

# Predict the test set sequences
predicted_sequences = model.predict([X_test, decoder_input_test])

# Convert predictions to label sequences
predicted_labels = np.argmax(predicted_sequences, axis=-1)

# Function to trim sequences by removing trailing zeros
def trim_sequences(sequences, pad_token=0):
    trimmed_sequences = []
    for seq in sequences:
        last_valid_index = 0
        for idx, token in enumerate(seq):
            if token != pad_token:
                last_valid_index = idx
        trimmed_seq = seq[:last_valid_index+1]
        trimmed_sequences.append(trimmed_seq)
    return trimmed_sequences

# Trim the predicted and true sequences for cleaner comparison
predicted_labels_trimmed = trim_sequences(predicted_labels)
y_test_trimmed = trim_sequences(y_test.argmax(axis=-1))

# Print out true and predicted sequences for comparison
num_sequences_to_print = 200
for i in range(num_sequences_to_print):
    true_seq = y_test_trimmed[i]
    pred_seq = predicted_labels_trimmed[i]
    
    print(f"True Sequence      {i+1}: {true_seq}")
    print(f"Predicted Sequence {i+1}: {pred_seq}\n")

39/39 [==============================] - 3s 48ms/step
True Sequence      1: [ 4 14  3]
Predicted Sequence 1: [4 5 3]

True Sequence      2: [ 3 15 14 19 20  5 12 12  1 20  9 15 14]
Predicted Sequence 2: [23 15 13 16 20  9  9  9  9  9  0 15 14]

True Sequence      3: [ 9 12 15 23]
Predicted Sequence 3: [ 2 12 15 23]

True Sequence      4: [14 15 21 14 19]
Predicted Sequence 4: [14  1 21 19 19]

True Sequence      5: [ 1  2 19 12]
Predicted Sequence 5: [20 20 19 20]

True Sequence      6: [12  9 14 11]
Predicted Sequence 6: [23  3  3  4]

True Sequence      7: [13 15 14 20  1  7  5 19]
Predicted Sequence 7: [ 1  1 14 20  0  0 19 19 20]

True Sequence      8: [6 2]
Predicted Sequence 8: [20  4  5]

True Sequence      9: [ 3  1 21 19  5]
Predicted Sequence 9: [ 3  1  8 14]

True Sequence      10: [16 18  1 14 11]
Predicted Sequence 10: [16 18  1 14  4]

True Sequence      11: [ 1 19 12]
Predicted Sequence 11: [16 19 20]

True Sequence      12: [12  9  5 12 19]
Predicted Sequence 12: [ 4  9

In [22]:
from tensorflow.keras.models import load_model

model_test = load_model(r"C:\Users\emrey\Desktop\Centennial\FOURTH SEMESTER\Software Development\Jones Model 2D\model_sign_2d_dim42.h5")

In [43]:
test_case = df['padded_normalized_landmarks_42'][0]

In [45]:
decoder_input_test = np.zeros_like(y_test)
decoder_input_test[:, 1:, :] = y_test[:, :-1, :]

In [52]:
display(X_test.shape,
decoder_input_test.shape)

(1238, 125, 42)

(1238, 17, 27)

In [63]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you have the following:
# predicted_labels_trimmed: The list of trimmed predicted sequences
# y_test_trimmed: The list of trimmed true sequences

# Flatten the lists of sequences to create a 1D array of labels
flat_predicted_labels = list(itertools.chain.from_iterable(predicted_labels_trimmed))
flat_true_labels = list(itertools.chain.from_iterable(y_test_trimmed))

# Compute the confusion matrix
cm = confusion_matrix(flat_true_labels, flat_predicted_labels)

# Assuming you have a LabelEncoder instance with the classes already fitted
# If your labels are numbers from 0 to 26, you can create the tick labels directly
tick_labels = [str(i) for i in range(27)]  # Adjust this range if you have a different number of classes

# Visualize the confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=tick_labels, yticklabels=tick_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


ValueError: Found input variables with inconsistent numbers of samples: [5035, 5005]

In [55]:
print(f"Shape of X_test: {X_test.shape}")
print(f"Type of X_test: {X_test.dtype}")

print(f"Shape of y_test: {y_test.shape}")
print(f"Type of y_test: {y_test.dtype}")

# Assuming you've already executed:
# decoder_input_test = np.zeros_like(y_test)
# decoder_input_test[:, 1:, :] = y_test[:, :-1, :]

print(f"Shape of decoder_input_test (after shifting): {decoder_input_test.shape}")

# Assuming your model is named 'model'
model_input_shapes = [model_input.shape for model_input in model.input]
print(f"Model's expected input shapes: {model_input_shapes}")

# Now, manually compare these shapes with the shapes of X_test and decoder_input_test you printed earlier.

# Selecting a single sample from X_test and decoder_input_test
single_X_test = np.expand_dims(X_test[0], axis=0)
single_decoder_input_test = np.expand_dims(decoder_input_test[0], axis=0)

# Making a prediction with the single sample
single_predicted_sequence = model.predict([single_X_test, single_decoder_input_test])

# Checking the shape of the predicted sequence
print(f"Shape of single predicted sequence: {single_predicted_sequence.shape}")

# Optionally, if you know the expected shape of the output, compare it here


Shape of X_test: (1238, 125, 21, 3)
Type of X_test: float64
Shape of y_test: (1238, 17, 27)
Type of y_test: float64
Shape of decoder_input_test (after shifting): (1238, 17, 27)
Model's expected input shapes: [(None, 125, 21, 3), (None, None, 27)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
Shape of single predicted sequence: (1, 17, 27)
